In [2]:
import random
import matplotlib.pyplot as plt
import math
import numpy as np
import matplotlib.patches as patches
random.seed(5918)
np.random.seed(5918)

In [8]:
%run Preprocessing.ipynb

In [9]:

def make_a_test_map(n_nodes:int):
    max_x = 100
    max_y = 100
    
    i = 1
    
    grid = {}
    
    grid[0] = (0,0)
    
    while i<=n_nodes:
        grid[i] = (random.randint(0,99),random.randint(0,99))
        i+=1
    
    return grid

def distance(x1,y1,x2,y2):
    
    return math.sqrt((x2-x1)**2+(y2-y1)**2)

def detour_length(p1,p2,c1,grid):
    x1,y1 = grid[p1]
    x2,y2 = grid[p2]
    x3,y3 = grid[c1]
    
    new_length = distance(x1,y1,x2,y2)+distance(x2,y2,x3,y3)
    old_length = distance(x1,y1,x3,y3)

    return new_length-old_length
    


def make_tour(grid:dict, tourlength:int):
    '''
    This function takes a dictionary of nodes and selects a given number of nodes to make a tour
    beginning and ending at the depot
    
    returns a list containing the order of the nodes, and the length of the tour
    '''
    tour = [0]
    dist = 0
    nodes = list(range(1,len(grid)))
    random.shuffle(nodes)
    curr_node = 0
    i = 1
    while i<=tourlength:
        new = nodes.pop()
        tour.append(new)
        x1, y1 = grid[curr_node]
        x2, y2 = grid[new]
        dist += distance(x1,y1,x2,y2)
        curr_node = new
        i+=1
    
    tour.append(0)
    x1, y1 = grid[curr_node]
    x2, y2 = grid[0]
    dist += distance(x1,y1,x2,y2)
      
    return tour, dist


In [40]:
def make_map_with_clusters(n_nodes_per_cluster:int, n_clusters:int,cluster_radius = 200):

    max_x = 1000
    max_y = 1000
    
    turbine_id = 1
    
    grid = {}
    
    grid[0] = (0,0)
    
    cluster_centers = []
    cs = []
    
    for _ in range(n_clusters):
        angle = random.uniform(0,math.pi/2)
        
        apexx = int(cluster_radius*math.cos(angle))
        apexy = int(cluster_radius*math.sin(angle))
        cx = round(random.uniform(apexx, max_x))
        cy = round(random.uniform(apexy,max_y))
        cluster_centers.append((cx,cy))
        grid[turbine_id] = (cx,cy)
        cs.append(turbine_id)
        turbine_id+=1
        
    for cluster in cluster_centers:
        cx,cy = cluster
    
        for _ in range(n_nodes_per_cluster):
            x = int(random.gauss(cx,50))
            y = int(random.gauss(cy,50))
            
            x = max(0,min(x,max_x))
            y = max(0,min(y,max_y))
            
            grid[turbine_id] = (x,y)
            turbine_id +=1
    

    return grid,cs,cluster_centers

    
    
    

In [41]:
def generate_jobs(nodes:dict,n_chargers): #The nodes here should only be the OTWs
    """This function generates jobs for the problem. The crew_needed for each job states how much of the passanger capacity the crew for this job takes in the vessel. The priority property is used
    in the objective function in the case that a job is left unserved. A higher priority yields a larger penalty. This function can be expanded to give more parameters to the jobs-

    Args:
        nodes (dict): All the nodes in the problem
        n_chargers (int): The number of chargers in the problem. This is used to find the correct index of the OTW-nodes

    Returns:
        jobs (dict): All the jobs with turbine_id as key and (crew_needed, priority) as value. 
    """
    
    n_nodes = len(nodes)
    n_jobs = n_nodes-(n_chargers+1)
    
    crew_needed = np.clip(np.random.normal(loc=5, scale=2, size=n_nodes).astype(int), a_min=1, a_max=None)
    crew_needed = crew_needed.tolist()

    
    jobs = {}
    for i in range(n_chargers+1,n_nodes):
        tag = i
        job = crew_needed.pop()
        jobs[tag] = job

    return jobs


In [42]:
'''test = make_a_test_map(10)
tour1, len1 = make_tour(test,3)
tour2, len2 = make_tour(test,2)
cluster_test, centers = make_a_test_map_with_clusters(3,3,35)
clustertour1, clustertourlength1 = make_tour(cluster_test,4)
clustertour2, clustertourlength2 = make_tour(cluster_test,3)
'''

'test = make_a_test_map(10)\ntour1, len1 = make_tour(test,3)\ntour2, len2 = make_tour(test,2)\ncluster_test, centers = make_a_test_map_with_clusters(3,3,35)\nclustertour1, clustertourlength1 = make_tour(cluster_test,4)\nclustertour2, clustertourlength2 = make_tour(cluster_test,3)\n'

In [45]:
'''clusterplot = show_the_grid(cluster_test,len(centers))
plot_tour_on_map(clusterplot,cluster_test,clustertour1,clustertourlength1,"Tour 1")
plot_tour_on_map(clusterplot,cluster_test,clustertour2,clustertourlength2,"Tour 2",color = 'purple')
plot_cluster_centers(clusterplot,centers,30)
print(clustertour2)'''

'clusterplot = show_the_grid(cluster_test,len(centers))\nplot_tour_on_map(clusterplot,cluster_test,clustertour1,clustertourlength1,"Tour 1")\nplot_tour_on_map(clusterplot,cluster_test,clustertour2,clustertourlength2,"Tour 2",color = \'purple\')\nplot_cluster_centers(clusterplot,centers,30)\nprint(clustertour2)'

In [46]:
def montecarlolength():
    '''
    I want this function to make 10 grids, and for each grid to make 10000 tours of tourlength tourlength.
    I should then check that each node is visited evenly, and primarily it should calculate the average length of the tours
    '''
    avgs = np.zeros(4)
    i_s = 10
    j_s = 100000
    for n in range(1,5):
        
        for i in range(i_s):
            
            curr_map = make_a_test_map(15)
            
            for j in range(j_s):
                tour, tourlength = make_tour(curr_map,n)
                avgs[n-1] += tourlength/(i_s*j_s)
    
    
    return avgs
        

In [47]:
def montecarlolength_cluster():
    '''
    I want this function to make 10 grids, and for each grid to make 10000 tours of tourlength tourlength.
    I should then check that each node is visited evenly, and primarily it should calculate the average length of the tours
    '''
    avgs = np.zeros(4)
    i_s = 10
    j_s = 100000
    for n in range(1,5):
        
        for i in range(i_s):
            
            curr_map,_, _ = make_map_with_clusters(6,1,20)
            
            for j in range(j_s):
                tour, tourlength = make_tour(curr_map,n)
                avgs[n-1] += tourlength/(i_s*j_s)
    
    
    return avgs

In [48]:
#lengths = montecarlolength()

In [49]:
#cluster_lengths = montecarlolength_cluster()

In [50]:
def vessel_types():
    vessels = {}
    #ID: The keys in the dictionary is the vessel id.
    battery_range = [2000,3000,4500] # I think these ranges should be okay for now since it allows the medium and big vessel type to go to more than one cluster
    #coeffecient = [] #kWh / km 
    operational_cost = [1, 2, 3.5] #cost per km
    speed = [20,22,25] # Knots. In km/h: 37,41,46
    passenger_capacity = [10,15,25] # This will be modified depending on the avg size of the crews.
    fixed_cost = [5000,10000,17000]
    
    
    vessels[1] = [1,battery_range[0],operational_cost[0],speed[0], passenger_capacity[0],fixed_cost[0]]
    vessels[2] = [2,battery_range[1],operational_cost[1],speed[1], passenger_capacity[1],fixed_cost[1]]
    vessels[3] = [3,battery_range[2],operational_cost[2],speed[2], passenger_capacity[2],fixed_cost[2]]
    
    return vessels
    
    
    

In [51]:
def all_nodes(jobs):
    
    nodes = []
    
    for job in jobs.keys():
        nodes.append((job,0))
        nodes.append((job,1))
    
    return nodes

In [3]:
def make_problem(n_nodes_per_cluster, n_chargers,print_=True):
    
    random.seed(5918)
    
    problem = {}
    
    locations, cs, cs_locations = make_map_with_clusters(n_nodes_per_cluster=n_nodes_per_cluster,n_clusters=n_chargers)
    problem["Locations"] = locations
    charging_stations = list(zip(cs,cs_locations))
    
    problem["ChargingStations"] = charging_stations
    
    
    jobs = generate_jobs(locations,len(charging_stations))
    problem["Jobs"] = jobs
    
    
    nodes = all_nodes(jobs)
    
    problem["Nodes"] = nodes
    
    vessels = vessel_types()
    problem["Vessels"] = vessels
    
    
    costs = {"Fixed Penalty": 2000, #Penalty for each crew member outsourced. This makes a bigger job be more prenalized than a smaller job
             "PenaltyPerCrew": 750
            }
    problem["Costs"] = costs
    
    pp = preprocessing(problem)
    
    problem["PreProseccing"] = pp
    
    if print_:
        print("Jobs Information:")
        print("{:<15} {:<15}".format("Turbine ID", "Crew Needed"))
        for turbine_id, (crew_needed) in jobs.items():
            print("{:<15} {:<15}".format(turbine_id, crew_needed))
        
        print("Vessel Types Information:")
        print("{:<10} {:<15} {:<20} {:<25} {:<20} {:<20}".format("Vessel ID", "Battery Range", "Operational Cost", "Speed (Knots)", "Passenger Capacity", "Fixed Cost"))
        for specs in vessels.values():
            vessel_id, battery_range, operational_cost, speed, passenger_capacity,fixed_cost = specs
            print("{:<10} {:<15} {:<20} {:<25} {:<20} {:<20}".format(vessel_id, battery_range, operational_cost, speed, passenger_capacity,fixed_cost))
    
        
    
    
    return problem
    